In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,Flatten,Dropout,Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Add,Activation,ZeroPadding2D,AveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.models import Model

In [2]:
trainDataGen=ImageDataGenerator(rescale=1./255,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True
                            )

In [3]:
testDataGen=ImageDataGenerator(rescale=1./255)

In [4]:
training_set=trainDataGen.flow_from_directory('Datasets/Train',
                                              target_size=(227,227),
                                              batch_size=40,
                                              class_mode='categorical'
                                                )

Found 25000 images belonging to 2 classes.


In [5]:
test_set=testDataGen.flow_from_directory('Datasets/test',
                                              target_size=(227,227),
                                              batch_size=40,
                                              class_mode='categorical'
                                                )

Found 1872 images belonging to 2 classes.


<img src='resnet.png'><img>

<img src='ResnetArch.jpg'></img>

In [6]:
def identityBlock(x,f,filters):
    f1,f2,f3=filters
    X_shortcut=x
    
    ##first layer
    x=Conv2D(filters=f1,kernel_size=(1,1),strides=(1,1),padding='valid')(x)
    x=BatchNormalization()(x)
    x=Activation('relu')(x)
    
    ## Second Layer 
    x=Conv2D(filters=f2,kernel_size=(3,3),strides=(1,1),padding='same')(x)
    x=BatchNormalization()(x)
    x=Activation('relu')(x)
    
    ## Third Layer
    x=Conv2D(filters=f3,kernel_size=(1,1),strides=(1,1),padding='valid')(x)
    x=BatchNormalization()(x)
    
    x=Add()([x,X_shortcut])
    
    return x
    

In [7]:
def ConvolutionBlock(x,f,filters,s=2):
    f1,f2,f3=filters
    X_shortcut=x
    
    ##first layer
    x=Conv2D(filters=f1,kernel_size=(1,1),strides=(s,s),padding='valid')(x)
    x=BatchNormalization(axis=3)(x)
    x=Activation('relu')(x)
    
    ## Second Layer 
    x=Conv2D(filters=f2,kernel_size=3,strides=1,padding='same')(x)
    x=BatchNormalization()(x)
    x=Activation('relu')(x)
    
    ## Third Layer
    x=Conv2D(filters=f3,kernel_size=1,strides=1,padding='valid')(x)
    x=BatchNormalization()(x)
    
    ## Convolution Block this is provided to make input  and output size of image to be same 
    ## This Whole function is used when input size is differennt from output size if both are equall we will use identity 
    ## Function
    
    X_shortcut=Conv2D(filters=f3,kernel_size=1,padding='valid',strides=s)(X_shortcut)
    X_shortcut=BatchNormalization()(X_shortcut)
    
    
    
    
    x=Add()([x,X_shortcut])
    x=Activation('relu')(x)
    
    return x
    

In [8]:
def Resnet50(input_shape=(224,224,3),classes=2):
    X_input=Input(input_shape)
    
    ## 
    x=ZeroPadding2D(padding=(3,3))(X_input)
    
    ##
    x=Conv2D(64,kernel_size=(7,7),strides=(2,2))(x)
    x=BatchNormalization()(x)
    x=Activation('relu')(x)
    x=MaxPooling2D((3,3),strides=(2,2))(x)
    
    ##230-7=223+1=224-3=221+1=222/2=111-3=109+1=110
    x=ConvolutionBlock(x,f=3,filters=[64,64,256],s=1)
    x=identityBlock(x,3,[64,64,256])
    x=identityBlock(x,3,[64,64,256])
    
    ## 
    x=ConvolutionBlock(x,f=3,filters=[128,128,512],s=2)
    x=identityBlock(x,3,[128,128,512])
    x=identityBlock(x,3,[128,128,512])
    x=identityBlock(x,3,[128,128,512])
    x=identityBlock(x,3,[128,128,512])
    
    ##
    x=ConvolutionBlock(x,f=3,filters=[256,256,1024],s=2)
    x=identityBlock(x,3,[256,256,1024])
    x=identityBlock(x,3,[256,256,1024])
    x=identityBlock(x,3,[256,256,1024])
    x=identityBlock(x,3,[256,256,1024])
    x=identityBlock(x,3,[256,256,1024])
    x=identityBlock(x,3,[256,256,1024])
    
    ##
    x=ConvolutionBlock(x,f=3,filters=[512,512,2048],s=2)
    x=identityBlock(x,3,[512,512,2048])
    x=identityBlock(x,3,[512,512,2048])
    
    ##
    
    x=AveragePooling2D((2,2),padding='same')(x)
    x=Flatten()(x)
    x=Dense(units=1000,activation='relu')(x)
    x=Dense(units=2,activation='softmax')(x)
    
    ## 
    model=Model(inputs=X_input,outputs=x)
    

    return model
    

In [9]:
model=Resnet50(input_shape=(224,224,3),classes=2)

# 

In [10]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 64) 9472        zero_padding2d[0][0]             
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 112, 112, 64) 256         conv2d[0][0]                     
______________________________________________________________________________________________

In [11]:
from tensorflow.keras.optimizers import Adam

In [12]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit(x=training_set,validation_data=test_set,epochs=5)

Epoch 1/5
